<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

In this lab, you will implement some of the techniques discussed in the lecture.

Below you are given a solution to the previous scenario. Note that it has two serious drawbacks:
 * The output predictions do not sum up to one (i.e. it does not return a distribution) even though the images always contain exactly one digit.
 * It uses MSE coupled with output sigmoid which can lead to saturation and slow convergence 

**Task 1.** Use softmax instead of coordinate-wise sigmoid and use log-loss instead of MSE. Test to see if this improves convergence. Hint: When implementing backprop it might be easier to consider these two function as a single block and not even compute the gradient over the softmax values. 

**Task 2.** Implement L2 regularization and add momentum to the SGD algorithm. Play with different amounts of regularization and momentum. See if this improves accuracy/convergence.

**Task 3 (optional).** Implement Adagrad, dropout and some simple data augmentations (e.g. tiny rotations/shifts etc.). Again, test to see how these changes improve accuracy/convergence.

**Task 4.** Try adding extra layers to the network. Again, test how the changes you introduced affect accuracy/convergence. As a start, you can try this architecture: [784,100,30,10]


In [ ]:
import random
import numpy as np
from torchvision import datasets, transforms

In [ ]:
# Let's read the mnist dataset

def load_mnist(path='.'):
    train_set = datasets.MNIST(path, train=True, download=True)
    x_train = train_set.data.numpy()
    _y_train = train_set.targets.numpy()
    
    test_set = datasets.MNIST(path, train=False, download=True)
    x_test = test_set.data.numpy()
    _y_test = test_set.targets.numpy()
    
    x_train = x_train.reshape((x_train.shape[0],28*28)) / 255.
    x_test = x_test.reshape((x_test.shape[0],28*28)) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1
    
    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def relu(z):
  return np.maximum(0, z)

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

In [ ]:
from scipy.special import softmax

# def softmax(x, axis=0):
#     return np.exp(x) / np.exp(x).sum(axis=axis)

EPSILON = 1e-8

class Network(object):
    def __init__(self, sizes, momentum=0.9, l2=0.01, drop_rate_input=0.9, drop_rate_hidden=0.5):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.momentum = momentum
        self.l2=l2

        assert 0 <= drop_rate_input <= 1
        assert 0 <= drop_rate_hidden <= 1
        self.drop_rate_input = drop_rate_input
        self.drop_rate_hidden = drop_rate_hidden

        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
        # momentum
        self.weights_momentum = [np.zeros_like(w) for w in self.weights]
        self.biases_momentum = [np.zeros_like(b) for b in self.biases]

        # adagrad
        self.weights_grad_var = [np.zeros_like(w) for w in self.weights]
        self.biases_grad_var = [np.zeros_like(b) for b in self.biases]

    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        for layer_index, (b, w) in enumerate(zip(self.biases, self.weights)):
            a = np.matmul(w, a)+b
            if layer_index < self.num_layers-2:
              a = sigmoid(a)
              # a = relu(a)
            else:
              a = softmax(a, axis=0)
        return a
    
    def update_mini_batch(self, mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch which is as in tensorflow API.
        # eta is the learning rate

        nabla_b, nabla_w = self.backprop(mini_batch[0].T,mini_batch[1].T)
        nabla_b = [b / len(mini_batch[0]) for b in nabla_b]
        nabla_w = [w / len(mini_batch[0]) for w in nabla_w]

        # Adagrad
        self.biases_grad_var = [bg + np.square(bm) for bg, bm in zip(self.biases_grad_var, nabla_b)]
        self.weights_grad_var = [wg + np.square(wm) for wg, wm in zip(self.weights_grad_var, nabla_w)]

        # l2 regularization
        self.weights = [w*(1-self.l2*eta) for w in self.weights]

        eta_ada_biases = [eta / np.sqrt(EPSILON + var) for var in self.biases_grad_var]
        eta_ada_weights = [eta / np.sqrt(EPSILON + var) for var in self.weights_grad_var]


        self.biases_momentum = [self.momentum*b + np.multiply(eta_ada, nb) for b, nb, eta_ada in zip(self.biases_momentum, nabla_b, eta_ada_biases)]
        self.weights_momentum = [self.momentum*w + np.multiply(eta_ada, nw) for w, nw, eta_ada in zip(self.weights_momentum, nabla_w, eta_ada_weights)]
            
        self.weights = [w-nw 
                        for w, nw in zip(self.weights, self.weights_momentum)]
        self.biases = [b-nb 
                       for b, nb in zip(self.biases, self.biases_momentum)]

        
    def backprop(self, x, y):
        # For a single input (x,y) return a pair of lists.
        # First contains gradients over biases, second over weights.
        g = x
        
        # dropout input
        mask = (np.random.rand(*g.shape) < self.drop_rate_input).astype(np.float32)
        g = mask * g / self.drop_rate_input

        gs = [g] # list to store all the gs, layer by layer
        fs = [] # list to store all the fs, layer by layer
        for layer_index, (b, w) in enumerate(zip(self.biases, self.weights)):
            f = np.dot(w, g)+b
            fs.append(f)
            if layer_index < self.num_layers-2:
              # dropout hidden layers
              mask = (np.random.rand(*g.shape) < self.drop_rate_hidden).astype(np.float32)
              g = mask * g / self.drop_rate_hidden
              
              g = sigmoid(f)
              # g = relu(f)
            else:
              g = softmax(f, axis=0)

            gs.append(g)

        # backward pass <- both steps at once
        a = g
        dLdg = self.cost_derivative(gs[-1], y)
        dLdfs = []
        for layer_index, (w,g) in reversed(list(enumerate(zip(self.weights,gs[1:])))):
            if layer_index < self.num_layers-2:
              dLdf = np.multiply(dLdg,np.multiply(g,1-g)) # sigmoid
              # dLdf = np.multiply(dLdg, (g>0) * 1.0) # relu
            else:
              dLdf = dLdg
            dLdfs.append(dLdf)
            dLdg = np.matmul(w.T, dLdf)
        dLdWs = [np.matmul(dLdf,g.T) for dLdf,g in zip(reversed(dLdfs),gs[:-1])] 
        dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)] 
        return (dLdBs,dLdWs)

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        best_acc = -1
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch((x_mini_batch, y_mini_batch), eta)
            if test_data:
                accuracy = self.evaluate((x_test, y_test))
                best_acc = max(best_acc, accuracy)
                print("Epoch: {0}, Accuracy: {1}".format(j, accuracy))
                print([np.sum(w) for w in self.weights])
            else:
                print("Epoch: {0}".format(j))
        print(f"Best accuracy: {best_acc}")


In [17]:
# network = Network([784,30,10], momentum=0.9, l2=1e-3)
# network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=0.1, test_data=(x_test, y_test))
network = Network([784,30,10], momentum=0.9, l2=1e-3, drop_rate_input=0.9, drop_rate_hidden=0.5)
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=1, test_data=(x_test, y_test))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Epoch: 0, Accuracy: 0.4461
[-21590.3548511706, 1620.8140329865937]
Epoch: 1, Accuracy: 0.695
[-11285.949717055655, 870.8722492970046]
Epoch: 2, Accuracy: 0.7261
[-8705.061162171265, 457.9677875507619]
Epoch: 3, Accuracy: 0.7833
[-4973.092811249503, 229.02893929898624]
Epoch: 4, Accuracy: 0.7773
[-3232.1153872955556, 103.73221514750895]
Epoch: 5, Accuracy: 0.7269
[-2689.471922590287, 37.66994804417607]
Epoch: 6, Accuracy: 0.8021
[-3353.1472292046597, -4.488933164668014]
Epoch: 7, Accuracy: 0.8165
[-2572.24940961351, -27.40615200551357]
Epoch: 8, Accuracy: 0.8347
[-2775.383759232089, -38.541166259070266]
Epoch: 9, Accuracy: 0.8324
[-3486.1432775980447, -34.265084489106584]
Epoch: 10, Accuracy: 0.8345
[-9117.90795544885, -26.45073882035868]
Epoch: 11, Accuracy: 0.8499
[-7208.5000497524225, -36.9934095198725]
Epoch: 12, Accuracy: 0.8412
[-5325.632801049166, -46.05225736061159]
Epoch: 13, Accuracy: 0.8556
[-6969.185378051735, -43.281729448186866]
Epoch: 14, Accuracy: 0.8681
[-9448.991385129